# Understanding our galaxy by using Gaia data
-----------------------------------------------------

Gaia spacecraft has thoroughly observed our galaxy for 2013-2025, and broadened our understanding of our galaxy's structure & dynamics by enormous magnitude. It has recently been retired, after observing 1 billion stars, 14 times every year. It has recorded the positions and motions of the stars in our galaxy with remarkable precision.  

Today we will explore Gaia data and try to learn about our galaxy and gain insights from the data.

There are four datasets that I will use: 

1. Stellar data in solar neighborhood (5000 stars with distance < 100 pc) obtained from Gaia Data Release 3 main source catalog by ADQL query
2. Stellar data in our galaxy (10,000 stars with distance < 10 kpc) obtained from Gaia Data Release 3 main source catalog by ADQL query
3. Catalog of open clusters: [Cantat-Gaudin et al. 2018](https://ui.adsabs.harvard.edu/abs/2018A%26A...618A..93C/abstract)
4. Catalog of globular clusters: [Vasilev & Baumgardt 2021](https://ui.adsabs.harvard.edu/abs/2021MNRAS.505.5978V/abstract)

The four datasets can be found in this [drive link](https://drive.google.com/drive/folders/1L-otdFP2CM5Rqm3C4iCBTdhV9sku4t8M?usp=sharing)


In [ ]:
#imports
# astropy imports
import astropy.coordinates as coord
from astropy.table import QTable
from astropy.table import Table
import astropy.units as u


# Other necessary imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# gala imports
import gala.dynamics as gd
import gala.potential as gp

Before we begin, a quick version check. Just so that you know what you're dealing with. My versions are: 

numpy: 1.26.4

matplotlib: 3.10.7

astropy: 7.1.1

astroquery: 0.4.11

gala: 1.10.1

scipy: 1.16.3

It is absolutely fine to have other versions, just make sure they don't conflict with each other. 

In [ ]:
import astropy, gala, matplotlib, scipy, astroquery
print('numpy:',np.__version__)
print('matplotlib:', matplotlib.__version__)
print('astropy:',astropy.__version__)
print('astroquery:',astroquery.__version__)
print('gala:', gala.__version__)
print('scipy:', scipy.__version__)

In [ ]:
!which python #check where all the packages are imported from

## Reading the data from voTable using astropy.table.Table

`Table` object allows you to read `.vot` files very efficiently. 

In [ ]:
stars_near= Table.read("gaia_data_100pc-result.vot")
stars_gal= Table.read("gaia_data_10kpc-result.vot")
op_cl= Table.read("open_clusters.vot")
gl_cl= Table.read("glob_clusters.vot")

Now let's see the Tables

In [ ]:
stars_near[:4]

In [ ]:
stars_gal[:4]

### Description of the Gaia stellar data table columns
----------------------------------------------------------
1. `SOURCE_ID`:          A unique identifier by Gaia
2.  `ra:`                 Right ascention in deg
3. `dec`:                 Declination in deg
4.  `parallax:`            Parallax in mas
5. `parallax_error`:        Error in parallax measurement in mas
6. `parallax_over_error:`    Parallax/ parallax_error. We have selected data where parallax> 10* parallax_error
7. `pmra`: Proper motion along right ascension, in mas/yr. As discussed in the class, this is actually $\mu_\alpha*=\dot{\alpha}\cos \delta$
8. `pmdec:` Proper motion in declination, in mas/yr ($\\mu_\delta$).
9. `radial_velocity`: Radial velocity in km/s
10. `phot_g_mean_mag:` G band () mean magnitude
11. `phot_bp_mean_mag`: Mean magnitude in Blue band photometer
12. `phot_rp_mean_mag:` Mean magnitude in Red band photometer
14. `bp_rp`: Color, simply `phot_bp_mean_mag- phot_r_p_mean_mag`. It can be used as a proxy for temperature. 
15. `has_rvs:` A boolean which says whether a particular source has a radial velocity spectra or not. 
16. `ag_gspphot`: Average photometric extinction in G band
17. `ebpminrp_gspphot:` Reddening factor, also known as the color excess.
18. `teff_gspphot`: Effective temperature from low resolution spectra 
19. `logg_gspphot:` Surface gravity (log g) from low resolution spectra
20. `mh_gspphot`: Metallicity (`[M/H]`) from low resolution spectra
21. `teff_gspspec:` Effective temp ($\mathrm{T_{eff}}$) derived from med resolution spectra (only availabel for `has_rvs=True`)
22. `logg_gspspec`: $\log g$ derived from med resolution spectra (only availabel for `has_rvs=True`)
23. `mh_gspspec`: Metallicity (`[M/H]`) derived from med resolution spectra (only availabel for `has_rvs=True`)

In [ ]:
op_cl[:4]

Which are RA, DEC, proper motions in RA & DEC, parallax and mode of distances to the individual stars in the cluster. We can take dmode as a proxy for distance here.

In [ ]:
gl_cl[:4]

## Using astropy.coordinates

`astropy.coordinates` provide a lot of useful functions to deal with position and velocity data of the stars. If measured parallax of a star is $\theta$, then the distance to the star is:
$$d=\dfrac{1}{\theta}$$
If the parallax is measured in mili-arcseconds (like in these datasets), then the distance in parsec would be (because 1 pc= 1 AU/ 1 arcsec):
$$d=\dfrac{1000}{\theta} \mathrm{pc}$$
The `coord.Distance` module lets you to do this inversion smoothly:

In [ ]:
#Getting distance from Parallax
#It is just inversion with proper units

dist_n= coord.Distance(parallax=stars_near["parallax"])
dist_g= coord.Distance(parallax=stars_gal["parallax"])

If your astropy version is old, the above may not work. In that case, try either of the following:

`dist_n=coord.Distance(parallax=u.Quantity(stars_near["parallax"]))`

or, do it manually, and multiply with the `astropy.units` object, u.pc:

`dist_n=(1000/stars_near["parallax"].value)*u.pc`

For open cluster dataset (`op_cl` Table), mode of the distances to the individual stars are given in the column `dmode`, so we are not inverting the parallax. (However, you can check how parallax inversions and `dmode` compare! They wont match, and think why.)

For globular cluster dataset (`gl_cl` Table), some of the parallaxes are negative. That means the true parallax is likely small compared to the uncertainty of the measurement. We will discard these data points.

In [ ]:
gl_cl[gl_cl['plx']<0]

We will discard the data for these 9 clusters. You see here `e_plx`, i,e, the error in parallax is quite large compared to `plx`. 

In [ ]:
#dist_gc= coord.Distance(parallax= gl_cl["plx"], allow_negative=True)
glc_d=gl_cl[gl_cl['plx']>0] #subset with plx>0
dist_gc=coord.Distance(parallax=glc_d['plx'])

Now we are ready to locate the stars!

For each star we have two angles $\alpha$ (RA) and $\delta$ (DEC), and the distance ($d$). In `astropy.coordinates`, the very handy function for this is `SkyCoord`, and create Sky coordinate objects for all 4 of our datasets.

In [ ]:
#3d coordinates of the nearby stars
st_n= coord.SkyCoord(ra=stars_near["ra"],
                      dec=stars_near["dec"],
                      distance=dist_n
                      )
#3d coordinates of the galactic stars
st_g= coord.SkyCoord(ra=stars_gal["ra"],
                      dec=stars_gal["dec"],
                      distance=dist_g
                      )
#3d coordinates of the open clusters
opc= coord.SkyCoord(ra=op_cl["RAJ2000"],
                     dec=op_cl["DEJ2000"],
                     distance=op_cl["dmode"]
                     )
#3d coordinates of globular clusters with distance measures
glc= coord.SkyCoord(ra=glc_d["RAJ2000"],
                     dec=glc_d["DEJ2000"],
                     distance=dist_gc
                     )

In [ ]:
st_n #print and check others as well!

(ICRS) written here refers to the International Celestial Reference System, which is adopted as the standard celestial coordinate system by IAU. Unless otherwise mentioned all coordinate objects follow ICRS. 

### Conversion to galactic coordinates is super easy!
-------------------------------------------------------------

As mentioned in the class, conversion to galactic coordinates from celestial, involves two matrix multiplication. One rotates the x & y axes, and the other flips the equatorial plane to the galactic plane. This whole process is super easy with `SkyCoord` object, all you need to do is:

In [ ]:
st_n.galactic

And voila, you're done! now you have l, b and distance instead of ra, dec & distance. 

Let's do it for all, nearby stars (`st_n`), stars distributed further out in galaxy (`st_g`), open clusters (`opc`), and globular clusters (`glc`). 

In [ ]:
st_n_gal= st_n.galactic
st_g_gal= st_g.galactic
opc_gal= opc.galactic
glc_gal= glc.galactic

For reference, let's get the coordinates of sun and galactic center as SkyCoord objects. You can look up the internet or Carrol & Ostille's book (page no 882), galactic center's coordinates in ICRS frame are: 

`ra = 17 h 45 m 37. 2 s `

`dec = -28 deg 56 m 9.6 s`

`distance = 8.1 kpc `

And sun is the center/ origin of the ICRS coordinate system (actually the origin is the barycenter of the solar system, but we will ignore the difference now.)

Creating these skycoordinate objects are also very easy.

In [ ]:
gc= coord.SkyCoord(ra='17h45m37.2s',dec='-28d56m9.6s', distance=8.1*u.kpc, frame='icrs') #coordinate of galactic center from book
sun= coord.SkyCoord(ra=0*u.deg, dec=0*u.deg, distance=0*u.pc, frame='icrs') #coordinate of the observer

Now let's convert them to the galactic coordinates!

In [ ]:
gc_gal= gc.galactic
sun_gal= sun.galactic

In [ ]:
gc_gal

Notice galactic center's l & b are small but not exactly (0,0). 

This is because the coordinates mentioned in the book are not very precise. Higher decimal places are ignored. Nevertheless, this is good enough for now. 

## Visualizing the data in equatorial and galactic coordinates: distributions of stars, open clusters and globular clusters 
-----------------------------------------------------------------------------------------------------------------------------

Earth's continents are distributed on a sphere. When we draw a map, we take projections from points on the sphere to a plane. We will do the same here. To visualize the stars on the sky, we take projections. Different projection routines are available as kwargs in matplotlib, and different ways exist to implememt them.

We will use `aitoff` projection throughout this tutorial. To read about this projection, see [aitoff wiki](https://en.wikipedia.org/wiki/Aitoff_projection). 

We will start with the 2D distribution (ra, dec) and (l,b), and ignore the third axis, i.e. the distance for the moment. 

To get RA & DEC from the coord objects (e.g., `st_n`), you just have to write:

`st_n.ra` & `st_n.dec`

Now RA goes from 0 to 360 degree, and 0 and 360 degrees are the same. But we can't show that on a plane, therefore, we will wrap it at 180 degree. And then convert into radian. 

declination is easier, it goes from +90 to -90 degrees. We will convert it to radians as well. 


### Distribution of nearby stars

Let's start with the stars in the solar neighborhood. These are our nearest stars, Many of them are visible with naked eye. How are they distributed on the sky?

#### ICRS (RA-DEC)

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})

ax.scatter(st_n.ra.wrap_at(180 * u.deg).radian, st_n.dec.radian, s=0.2)
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r') # The galactic center
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

The "vertical" lines aka the longitudes are RA, and the "horizontal" lines aka the lattitudes are DEC. If you comment out the lines:

`ax.set_xticklabels([])`

`ax.set_yticklabels([])`

You will see the degree labels appear on the plot. However, I find them slightly off-putting, so I have removed the labels. 

The blue dots are all the 5000 stars in our neighborhood. They are quite evenly distributed on the sky. The red dot is the galactic center! 

Now let's see, if we plot the galactic coordinates of the same, what is the difference?

Here we will plot l vs b, so just have to get `st_n_gal.l` and `st_n_gal.b`

#### Galactic (l,b)

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})

ax.scatter(st_n_gal.l.wrap_at(180 * u.deg).radian, st_n_gal.b.radian, s=0.2)
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='r') #galactic center in galactic coordinates!
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

Mehh, the difference is not really appealing, on first glance it looks the same. The blue dots are everywhere. 

But wait! The galactic center, the red dot is now moved to the center! Of course it would, it is the galactic coordinates after all! 

But to really appreciate the galactic structure, and coordinate transformation, we need to look a bit further. 

-----------------------------------------------------------------------------------------------------------------

Before that, let's use the third axis ($d$) and get the cartesian coordiantes of these stars. 

$$x=d \cos\alpha\cos\delta,~ y=d \sin \alpha \cos \\delta,~ z = d \sin \delta$$

With SkyCoord object, in built frames are super convenient and you dont have to do these by hand,

All you need to do is `st_n_gal.cartesian.x`, `st_n_gal.cartesian.y`, `st_n_gal.cartesian.z`

Why am I using `st_n_gal` instead of `st_n`?

Because now my xy plane is the galactic plane itself! The xy plane in celestial coordinates is just extension of Earth's equatoirial plane, nothing special in galactic context. 

But with these, you get the distribution of the stars on the galactic plane (xy), and above and below the plane (yz) and (xz).

#### Cartesian (xyz)

In [ ]:
st_nx,st_ny,st_nz=st_n_gal.cartesian.x, st_n_gal.cartesian.y, st_n_gal.cartesian.z

fig, ax = plt.subplots(1,3, figsize=(12,3), dpi=300)
ax[0].scatter(st_nx, st_ny, s=0.5)
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[1].scatter(st_nx, st_nz,s=0.5)
ax[1].set_xlabel('x')
ax[1].set_ylabel('z')
ax[1].set_yticklabels([])
ax[2].scatter(st_ny, st_nz, s=0.5)
ax[2].set_xlabel('y')
ax[2].set_ylabel('z')
ax[2].set_yticklabels([])

Maaan, it's super boring! Everything looks the same! A blob of blue dots. 

But let's look further now. 10,000 stars within 10 kpc. How are they distributed?

### Distribution of stars within 10 kpc

Remember, coorinates for these stars are stored in `st_g`. For plotting we will do the same thing as above, just change `st_n` to `st_g`

#### ICRS (RA-DEC)

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})

ax.scatter(st_g.ra.wrap_at(180 * u.deg).radian, st_g.dec.radian, s=0.2)
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

Whoa, that's cool! 

See that band of stars? That's our galaxy. That's the milky way!

From different locations on Earth we have access to a different section of RAs and DECs, and we see only part of that band. 

Notice that most of the stars lie on that band, i.e., on the galactic plane. The number of stars above and below that plane is small. 

Before converting to galactic coordinates, let's color them according to distance. 

For that we need to use a colormap. One of my fav is `viridis`. 

In [ ]:
fig, ax =plt.subplots(figsize=(6,3),dpi=300, subplot_kw={'projection': 'aitoff'})
im=ax.scatter(st_g.ra.wrap_at(180 * u.deg).radian, 
              st_g.dec.radian, 
              s=0.2, 
              c=st_g.distance, 
              cmap='viridis', 
              vmin=st_g.distance.min().value,
              vmax=st_g.distance.max().value
             )
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)
cbar=fig.colorbar(im,ax=ax,shrink=0.7)
cbar.ax.set_ylabel("Distance(pc)")

Here purple corresponds to nearby stars and towards yellow are the stars further away. Notice that dark color is over represented. That's mainly because, since they are nearby, they are easier to detect by the instrument's camera. Far away, only the lights from the bright stars reach us. 

Now let's convert to Galactic coordinates! 

#### Galactic

In [ ]:
fig, ax =plt.subplots(figsize=(6,3),dpi=300, subplot_kw={'projection': 'aitoff'})
im=ax.scatter(st_g_gal.l.wrap_at(180 * u.deg).radian, 
           st_g_gal.b.radian, 
           s=0.2,
           c=st_g_gal.distance, 
           cmap='viridis', 
           vmin=st_g_gal.distance.min().value,
           vmax=st_g_gal.distance.max().value
          )
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)
cbar=fig.colorbar(im,ax=ax,shrink=0.7)
cbar.ax.set_ylabel("Distance(pc)")

The band got straightened ! As you expected. This is the galactic plane after all. And the red dot at the center is the galactic center. The stars far above the plane with low number density are the halo stars. 

#### cartesian (xyz)

In [ ]:
st_gx,st_gy,st_gz=st_g_gal.cartesian.x, st_g_gal.cartesian.y, st_g_gal.cartesian.z

fig, ax = plt.subplots(1,3, figsize=(14,3), dpi=300)
ax[0].scatter(st_gx, st_gy, s=0.5)
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[1].scatter(st_gx, st_gz,s=0.5)
ax[1].set_xlabel('x')
ax[1].set_ylabel('z')
#ax[1].set_yticklabels([])
ax[2].scatter(st_gy, st_gz, s=0.5)
ax[2].set_xlabel('y')
ax[2].set_ylabel('z')
ax[2].set_yticklabels([])

Notice that the number density of stars quickly drops with z, compared to x & y. You can also check that by plotting histograms in x y & z. 

### Distribution of open clusters
-----------------------------------------------------
Open clusters are young group of stars typically with age range from a few Myrs to few Gyrs. They are typically metal rich (lot of supernova in the past has made the environement rich in metal, and they are forming from it). They are usually foun on the galactic plane, where star formation rate is high.

Let's see where do our list of open clusters live! 


Note: If you remember, in the class we discussed how Trumpler in 1930 observed the angular diameter and fluxes of the open clusters. As a function of distance, angular diameter should fall as $1/d$, and flux should fall as $1/d^2$, so in a flux vs angular diameter lolog plot, you expect a straight line with slope 2 would fit the data. But Trumpler found that is not the case. Far away clusters (known from angular diameter) are fainter than expected! He concluded there must be something in between which is obscuring the light (like fog). 

#### ICRS (RA-DEC)

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})
ax.scatter(opc.ra.wrap_at(180 * u.deg).radian, opc.dec.radian, s=2,alpha=0.5)
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

You see, they are all (most of them) distributed on the galactic plane! That's why open clusters far away suffer so much extinction! Now let's transform these into galactic coordinates. Also let's color them with distances.

#### Galactic (l-b):

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})
im=ax.scatter(opc_gal.l.wrap_at(180 * u.deg).radian, opc_gal.b.radian,s=1,
           c=opc_gal.distance, 
           cmap='viridis', 
           vmin=opc_gal.distance.min().value,
           vmax=opc_gal.distance.max().value,
           )
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='r')
#ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)
cbar=fig.colorbar(im,ax=ax,shrink=0.7)
cbar.ax.set_ylabel("Distance(pc)")

In [ ]:
plt.hist(opc_gal.distance.value, bins=np.logspace(2,4.2,10))
plt.xscale('log')

Notice that most of the open clusters in this dataset are about 2-4 kpc distance away from us.

### Distribution of globular clusters
------------------------------------------

#### ICRS (RA-DEC)

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})
ax.scatter(glc.ra.wrap_at(180 * u.deg).radian, glc.dec.radian, s=2)
#ax.scatter(gld.ra.wrap_at(180 * u.deg).radian, gld.dec.radian, s=1, alpha=0.5)
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

How pecuiliar! You see, a bunch of globular clusters are clumped around the Galactic center! 

Harlow Shapley saw the same, and thought the same, how pecuiliar! Why are they clumped around that point towards the constellation of Sagittarius! How strange! 

And he studied the motion of the globular clusters carefully for a long time, and concluded, there must be the center of the galaxy! 

#### Galactic (l-b)

Now let's transform the coordinates to galactic.

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})
im=ax.scatter(glc_gal.l.wrap_at(180 * u.deg).radian, glc_gal.b.radian, s=2,c=glc_gal.distance, 
           cmap='viridis', 
           vmin=glc_gal.distance.min().value,
           vmax=glc_gal.distance.max().value)
#ax.scatter(gld.ra.wrap_at(180 * u.deg).radian, gld.dec.radian, s=1, alpha=0.5)
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)
cbar=fig.colorbar(im,ax=ax,shrink=0.7)
cbar.ax.set_ylabel("Distance(pc)")

In [ ]:
gc_y

In [ ]:
plt.hist(glc_gal.distance.value,bins=np.logspace(3.,4.2,10))
plt.xscale('log')
plt.xlabel('Distance(pc)')

See their distance is peaking at about 8 kpc. Shapley thought it was 15 kpc. But now our measurements got better. 

We can play with globular clusters more, but for now, let's look again at the stellar datasets.

## Inspecting the stars
-------------------------------
Let's remind ourselves what we have in the data


### Nearby stars

In [ ]:
stars_near[:4]

#### Color-magnitude diagram
---------------------------------------
Two columns are of particular interest. `bp_rp` and `phot_g_mean_mag`. 

Gaia has a broad band CCD used for astrometry, which collects photon in the wavelength range ~ 330-1050 nm, usually referred as G band. The received flux ($F_G$) is then converted to apparent magnitude ($m_G$): 

$$m_{G}=-2.5 \log(\dfrac{F_G}{F_Z})$$

Gaia's zero point flux used for calibration is: $F_Z \sim 2.5*10^{-11} \mathrm{erg s^{-1} cm^{-2} A^{-1}}$, which is approximately flux from a Vega like star at 10 pc distance.

And this apparent magnitude in G band, is stored in `phot_g_mean_mag`. 

On the other hand, Gaia has two photometers. Blue and red. They operate in ~ 330-680 nm, and ~ 630-1050 nm respectively. They record low resolution spectra for each source, and also mean magnitude in these photometric bands. When you subtract them, it gives you a slope in Spectral Energy Distribution. This is stored in `bp_rp` column. This slope can be used as a proxy for temperature. In Astronomy, this is often called **color**. 

So we have **magnitude**, as a proxy for **luminosity**, and **color** as a proxy for **temperature**. 

Therefore, we can construct a **color-magnitude diagram**, which is a proxy for **Hertsprung-Russel (HR) diagram**, and can characterize the stars in our dataset. 

Before that, first let's convert the apparent magnitude to the absolute magnitude. Since, $m_G=-2.5\log\left(\dfrac{L/4\pi d^2}{L_Z/ 4 \pi (10 \mathrm{pc})^2}\right)$, we can write:

$$M_G= m_G-5(\log(d)-1)$$

Where $M_G=-2.5\log\left( \dfrac{L}{L_Z} \right)$ is the absolute magnitude. 

Note: All logs are 10 based

The quantity $5(\log d -1)$ is known as the distance modulus. 

Again, SkyCoord object has a cool feature, you can just do `dist_n.distmod` to get the distance modulus. However, for older versions of Astropy it may not be available. In that case use the above formula. 

#### Getting the absolute mag and color-Nearby stars

In [ ]:
bp_rp_n= stars_near['bp_rp']
MG_n= stars_near['phot_g_mean_mag']-dist_n.distmod

#### Plotting the Color-magnitude diagram (CMD)-Nearby stars

In [ ]:
fig, ax =plt.subplots(figsize=(4,3), dpi=300)
ax.scatter(bp_rp_n, MG_n, s=0.2, alpha=0.5)
ax.set_xlabel("$G_{BP}-G_{RP}$")
ax.set_ylabel("$M_G$")
ax.invert_yaxis() # upward in y, increasing luminosity, decreasing magnitude
ax.set_title("HR diagram for 5000 stars within 100 pc")

Can you identify the main-sequence stars in the figure? Where are the giants? 

Note: Don't forget, these are only 5000 stars within 100 pc. 

Note: Note that, we have ignored extinction here, since they are within 100 pc, they won't suffer much extinction, that's what I thought. Is that true? please double check. (See below for reference)

### Galacric stars- stars within 10 kpc
-------------------------------------------

In [ ]:
stars_gal[:4]

#### Getting the absolute magnitude & color

In [ ]:
bp_rp_g= stars_gal['bp_rp']
MG_g= stars_gal['phot_g_mean_mag']-dist_g.distmod

#### Plotting the color-magnitude diagram

In [ ]:
fig, ax =plt.subplots(figsize=(4,3), dpi=300)
ax.scatter(bp_rp_g, MG_g, s=0.2, alpha=0.5)
ax.set_xlabel("$G_{BP}-G_{RP}$")
ax.set_ylabel("$M_G$")
ax.invert_yaxis()
ax.set_title("HR diagram for 10000 stars within 10 kpc")

The HR diagram above looks weird. One important thing we are forgetting, i.e. extinction by dust. We are looking at stars at ~kpcs order of distance, there's a lot of dust between the stars and us, which absorbs and scatters the light.


#### Extinction correction
----------------------------------------
Dust extinction has two major effects. The stars get fainter, and dust grain preferentially absorb and scatter bluer part of the light, hence the starlight gets redder.

Mathematically, if we express dust absorption coefficient with $\alpha_\nu$, when light travels through dust by a distance $ds$, it's intensity decreases by $\alpha_\nu ds$, therefore:

$$\dfrac{dI_\nu}{ds}=-\alpha_\nu I_\nu$$
Therefore, after travelling a distance s, the intensity is, integrating: 
$$I_\nu(s)=I_\nu(s_0)e^{-\int^s_{s_0} \alpha(s')ds'}=I_\nu e^{-\tau_\nu}$$

Therefore, $$ \tau_\nu= -\mathrm{ln} (I_\nu / I_{\nu_0})= -2.303 \log (F_\nu/ F_{\nu_0}) $$

Defining $A_\nu = 1.086 \tau_\nu$ so that $A_\nu = -2.5 \log (F_\nu/ F_{\nu_0})$

$A_\nu$ measures how much extinction in **magnitude** has light at frequency $\nu$ suffered. 

$A_\nu$ depends on the composition of dust.



---------------------------------

In Gaia dataset the G band extinction coefficient ($A_G$) and the excess reddenning factor ($E(BP-RP)=A_{BP}-A_{RP}$) are given in `ag_gspphot` and `ebpminrp_gspphot` respectively. 

Therefore, to get the correct absolute magnitude ($M_{G_\mathrm{corr}}$) and $({BP}-{RP})_\mathrm{corr}$ we need to do:

$$M_{G_\mathrm{corr}}= M_G - A_G $$

$$({BP}-{RP})_\mathrm{corr}= (BP-RP) - E(BP-RP) $$

$(M_G)_{\mathrm{corr}}$ and $(BP-RP)_\mathrm{corr}$ are the absolute magnitude and color if there were no absorption and scattering of photons by dust. 

In [ ]:
bp_rp_g_corr=bp_rp_g-stars_gal['ebpminrp_gspphot']
MG_g_corr=MG_g-stars_gal['ag_gspphot']

#### Extinction corrected CMD

In [ ]:
fig, ax =plt.subplots(figsize=(4,3), dpi=300)
ax.scatter(bp_rp_g_corr, MG_g_corr, s=0.2, alpha=0.5)
ax.set_xlabel("$G_{BP}-G_{RP}$")
ax.set_ylabel("$M_G$")
ax.invert_yaxis()
ax.set_title("Extinction corrected HR diagram")

To quantify extinction as a function of distance let's plot $A_G$ and $E(BP-RP)$ as a function of distance. It's natural to suspect more distant stars would suffer from more extinction. 

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(10,3),dpi=300)
ax[0].scatter(dist_g,
              stars_gal['ebpminrp_gspphot'],
             s=0.5,
             alpha=0.5)
ax[0].set_xscale('log')
ax[0].set_xlabel("Distance (pc)")
ax[0].set_ylabel("E(BP-RP)")
ax[1].scatter(dist_g, 
              stars_gal['ag_gspphot'],
             s=0.5,
             alpha=0.5)
ax[1].set_xscale('log')
ax[1].set_xlabel("Distance (pc)")
ax[1].set_ylabel("$A_G$")

We see that there is a lot of scatter, but the stars at the largest distances suffer the most extinction and reddening, as we suspected.


### Thin disk, Thick disk and stellar halo, who lives where
------------------------------------------------------------------

In class we discussed that the young and metal rich stars live in the thin disk, slightly older stars live in the thick disk, and very metal poor and very old stars live in the Halo. We also mentioned, high mass stars in the main-sequence, since their main sequence lifetime is short, are typically young. On the other hand, main sequence lifetime of low mass stars are incredibly long. Therefore, if we select a bunch of high mass stars, there's high probability that majority of them would be young. If we pick up a bunch of low mass stars, good fraction of them will be old. 

In this section we will do the following: 

1. Plot the z distribution of the stars. i.e. how above and below the galactic plane are they distributed.
2. Then we will several regions of z and overplot them on the HR diagram, and find their metallicity. 

----------------------------------------------------------------------------

Carrol & Ostille page 883 mentions scale height of thin disk is ~ 350 pc, for thick disks scale height is ~ 1000 pc, and Halo stars are beyond scale height of 1000 pc . Let's see what kind of stars are they! We will pick them up and overplot them on the HR diagram and get their metallicity. 

Obviously, the dataset we will be working with the galactic stars within 10 kpc dataset (`st_g` coord object, and `st_gal` Table )

The z coordinates are stored in st_gz object. 

In [ ]:
fig, ax =plt.subplots(figsize=(4,3),dpi=300)
ax.hist(np.abs(st_gz), bins=30, histtype='step',density=True)
ax.set_xlabel("|z| in pc")

In [ ]:
st_g.cartesian.z.value

In [ ]:
st_g_gal_thin=st_g_gal[np.abs(st_g_gal.cartesian.z.value)<350] # thin disk
st_g_gal_halo= st_g_gal[np.abs(st_g_gal.cartesian.z.value)>1000] # thick disk
st_g_gal_thick= st_g_gal[(np.abs(st_g_gal.cartesian.z.value)<1000) & (np.abs(st_g_gal.cartesian.z.value)>350)] # Halo stars

In [ ]:
fig, ax =plt.subplots(figsize=(6,3),dpi=300, subplot_kw={'projection': 'aitoff'})
ax.scatter(st_g_gal_thin.l.wrap_at(180 * u.deg).radian, 
           st_g_gal_thin.b.radian, 
           s=0.2,
           c='b',
           label='thin'
          )
ax.scatter(st_g_gal_thick.l.wrap_at(180 * u.deg).radian, 
           st_g_gal_thick.b.radian, 
           s=0.2,
           c='r',
           label='thick'
          )
ax.scatter(st_g_gal_halo.l.wrap_at(180 * u.deg).radian, 
           st_g_gal_halo.b.radian, 
           s=2,
           c='g',
           label='halo'
          )
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='k')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)
ax.legend(fontsize=8, bbox_to_anchor=(1.1,1.1))

Let's overplot them on the HR diagram. 

In [ ]:
mask_thin=(np.abs(st_g_gal.cartesian.z.value)<350)
mask_halo=(np.abs(st_g_gal.cartesian.z.value)>1000)
mask_thick= ((np.abs(st_g_gal.cartesian.z.value)<1000) & (np.abs(st_g_gal.cartesian.z.value)>350))

bp_rp_thin, bp_rp_halo, bp_rp_thick = bp_rp_g_corr[mask_thin], bp_rp_g_corr[mask_halo], bp_rp_g_corr[mask_thick]
MG_thin, MG_halo, MG_thick = MG_g_corr[mask_thin], MG_g_corr[mask_halo], MG_g_corr[mask_thick]

fig, ax =plt.subplots(figsize=(4,3), dpi=300)
#ax.scatter(bp_rp_g_corr, MG_g_corr, s=0.2, alpha=0.5)
ax.scatter(bp_rp_thin, MG_thin, s=0.1, alpha=0.5,c='b', label='thin')
ax.scatter(bp_rp_thick, MG_thick, s=0.1, alpha=0.5,c='r', label='thick')
ax.scatter(bp_rp_halo, MG_halo, s=0.1, alpha=0.5,c='g', label='halo')
ax.set_xlabel("$G_{BP}-G_{RP}$")
ax.set_ylabel("$M_G$")
ax.invert_yaxis()
ax.set_title("Thin & thick disk & halo stars on CMD")
ax.legend()

The picture suggests, the giants in the diagram, live in the Halo and thick disks, the majority of the main sequence stars shown here (mostly F,G,K even higher mass like A, B) are residents of thin disks! They are more evolved and older population than the main sequence stars.

Thick disks also may have low mass old stars, but they are underrepresented in this dataset. Can you think of a reason?

**Question: Why do you think the high mass are large in number in stars_gal and low in stars_near table? What type of stars are more numerous in the galaxy, low or high mass?**

Now let's plot the metallicity distribution of thin disk, thick disk and Halo stars. 

In [ ]:
fig, ax =plt.subplots(figsize=(4,3),dpi=300)

for mask, color, label in zip([mask_thin, mask_thick, mask_halo],['b','r','g'], ['thin', 'thick','Halo']):
    samp=stars_gal[mask]
    ax.hist(samp['mh_gspspec'], bins=30, histtype='step',density=True, color=color, label=label)
ax.set_xlabel("[M/H] (low-res)")
ax.legend()
ax.set_title("Metallicity distribution of stars in galaxy")

From this naive analysis, it seems that thin disk stars are more metal rich than thick disk and Halo stars, as mentioned in the book!

Now you can do the opposite, select metal rich and metal poor stars and plot their distribution in the galaxy.

## Using velocity information with coordinates

So far, we were only using position information. It turns out, we can also pack velocity info inside SkyCoord objects. 

Note: The `pmra` mentioned in Gaia catalog is actually $\mu_\alpha \cos \delta$, therefore while packing the info to SkyCoord object, we need to write pm_ra_cosdec instead of just pm_ra. 

Now, let's explore kinematics of the stars in our galaxy!

In [ ]:
#3d coordinates & v of the nearby stars
st_n= coord.SkyCoord(ra=stars_near["ra"],
                      dec=stars_near["dec"],
                      distance=dist_n,
                      pm_ra_cosdec=stars_near["pmra"],   #see the note above
                      pm_dec=stars_near["pmdec"],
                      radial_velocity=stars_near["radial_velocity"]
                      )
#3d coordinates & v of the galactic stars
st_g= coord.SkyCoord(ra=stars_gal["ra"],
                      dec=stars_gal["dec"],
                      distance=dist_g,
                      pm_ra_cosdec=stars_gal["pmra"], #see the note above
                      pm_dec=stars_gal["pmdec"],
                      radial_velocity=stars_gal["radial_velocity"]
                      )
#3d coordinates of the open clusters
#opc= coord.SkyCoord(ra=op_cl["RAJ2000"],
#                     dec=op_cl["DEJ2000"],
#                     distance=op_cl["dmode"],
#                     
#                     )

In [ ]:
st_n

### Conversion to galactic coords
--------------------------------------

Now with the position, velocity vectors also get transformed to new coordinates. 

Note: If it is not mentioned otherwise, all velocities are w.r.t solar system's barycenter. 

In [ ]:
st_n_gal=st_n.galactic
st_g_gal=st_g.galactic

In [ ]:
st_g_gal

### Stellar motion in solar neighborhood
----------------------------------------------

Getting the U,V,W components (radial, azimuthal and along z axis) is also super easy. We just need to write

`st_n_gal.velocity.d_xyz`

Just make sure the coord object, here `st_n_gal` is in galactic coordinates. 

This velocities are w.r.t solar barycenter.

In [ ]:
U,V,W = st_n_gal.velocity.d_xyz
fig, ax =plt.subplots(1,3, figsize=(15,3), dpi=300)
ax[0].scatter(V,U,s=0.2)
ax[0].scatter(0,0,s=5,c='k') 
ax[0].set_xlabel("V")
ax[0].set_ylabel("U")
ax[1].scatter(U,W,s=0.2)
ax[1].scatter(0,0,s=5,c='k')
ax[1].set_xlabel("U")
ax[1].set_ylabel("W")
ax[2].scatter(V,W,s=0.2)
ax[2].scatter(0,0,s=5,c='k')
ax[2].set_xlabel("V")
ax[2].set_ylabel("W")

### Local Standard of Rest & Pecuiliar velocity of sun:
--------------------------------------------------------

*Dynamic Local standard of rest* or simply the *Local standard of Rest (LSR)* is defined as a reference frame, which is located at a constant distance ($R_0$~8.1 kpc) from the center of the galaxy and rotating about the center of the galaxy with an uniform angular velocity in a perfect circle of radius $R_0$. The radial and z component of LSR velocity are zero. 

Sun is not exactly doing that, it has a motion w.r.t LSR. Relative velocity between the sun and LSR is called sun's **pecuiliar velocity**, and it's motion simply *solar motion*. 

**But sitting on Earth, how do we measure sun's peculiar velocity?**

The idea is, U & W are perturbative components added to the stars over their rotational velocities at random. Then, at any given moment, w.r.t LSR, the number of stars going upward the galactic plane, and the number of stars going upward with speed W and the number of stars going downward with speed W are roughly equal. Therefore, from LSR if we measure z component of velocities (W) of N number of stars, the average z component velocity should be zero. 

Same is the case for the radial component (U). The number of stars moving inward, would be roughly equal to the number of stars moving outward. Hence the average radial velocity w.r.t LSR should be zero.

But we are not on LSR, we are moving with the sun. Therefore, if we observe radial and z component of velocities of a bunch of stars and take the average, we get radial & z components of sun's pecuiliar velocity with a minus sign. 

Suppose, $(U, V, W)$ are the observed velocities w,.r.t sun. $(U_\mathrm{LSR}, V_\mathrm{LSR}, W_\mathrm{LSR})$ is the velocity of these stars w.r.t LSR, and $(U_\odot, V_\odot, W_\odot)$ is the sun's pecuiliar velocity. 
Mathematically: 

$$U=U_\mathrm{LSR}-U_\odot$$
$$V= V_\mathrm{LSR}- V_\odot$$
$$W= W_\mathrm{LSR}- W_\odot $$

Therefore, if we take average, $\langle U_\mathrm{LSR} \rangle=0$, and $\langle W_\mathrm{LSR} \rangle=0$, and: 

$$ U_\odot = - \langle U \rangle $$
$$ W_\odot = - \langle W \rangle $$


Measuring $V_\odot$ is tricky. Because, it turns out: $\langle V_\mathrm{LSR} \rangle < 0$ (See carrol & Ostille Page 905, and section 6.3 and section 7.6.2 in Arnab's book)

Assumin stellar motion is the result of a perturbation over uniform circular motion, and keeping the terms upto second order, one can show: 

$$\langle V_\mathrm{LSR} \rangle = C \langle U^{2} \rangle$$ Where C is a constant(See 7.6.2 in Arnab's book for a proof)

And, 

$$ V_\odot= \langle V \rangle - C \langle U^{2} \rangle $$

if we plot $\langle V \rangle$ vs $\langle U^{2} \rangle$ for different bunch of stars, the intercept should give us the azimuthal component of sun's pecuiliar velocity. 

In Carrol & Ostille as well as Arnab's book you will find; 

$$U_\odot = -10.0 ~\mathrm{km ~ s^{-1}}, ~ V_\odot = 5.2  ~\mathrm{km ~ s^{-1}}, ~ W_\odot = 7.2 ~\mathrm{km ~ s^{-1}}$$

These are the values adopted from [Binney & Merrifield 1998](https://ui.adsabs.harvard.edu/abs/1998gaas.book.....B/abstract), based on *Hipparcous* observations, and using a left handed coordinate system.

However, these values are revised now, and often people use right handed coordinate system nowadays (In Astropy Sky coords as well! ). The standard values for solar motion from [Schonrich et al 2010](https://ui.adsabs.harvard.edu/abs/2010MNRAS.403.1829S/abstract): 

$$U_\odot = 11.1^{+0.69}_{-0.75} ~\mathrm{km ~ s^{-1}}, ~ V_\odot = 12.24^{+0.47}_{-0.47}  ~\mathrm{km ~ s^{-1}}, ~ W_\odot = 7.25^{+0.37}_{-0.36} ~\mathrm{km ~ s^{-1}}$$

The sign flip in U is because of going from left handed to right handed coordinate system. 

-----------------------------------------------
Enough yapping, now let's see what our data tells us!

From our discussion above, U & W components of solar motion would be:

In [ ]:
print(f"U_sun: {-np.mean(U)}, W_sun: {-np.mean(W)}")

We are not very far off! (Instead of 11.1 and 7.25 we got 9.81 and 8.28 for our sample).

### Stellar motion within 10 kpc

In [ ]:
U,V,W = st_g_gal.velocity.d_xyz
fig, ax =plt.subplots(1,3, figsize=(14,3), dpi=300)
ax[0].scatter(V,U,s=0.2)
ax[0].scatter(0,0,s=5,c='k')
ax[0].set_xlabel("V")
ax[0].set_ylabel("U")
ax[1].scatter(U,W,s=0.2)
ax[1].scatter(0,0,s=5,c='k')
ax[1].set_xlabel("U")
ax[1].set_ylabel("W")
ax[2].scatter(V,W,s=0.2)
ax[2].scatter(0,0,s=5,c='k')
ax[2].set_xlabel("V")
ax[2].set_ylabel("W")

In [ ]:
#Solar motion for this sample:
print(f"U_sun: {-np.mean(U)}, W_sun: {-np.mean(W)}")

$\langle V \rangle$ not being zero is reflected in the asymmtery along the V axis in the above plot. Read about it from Carrol & Arnab's book. If time permits later I will add more on this. 

In [ ]:
fig, ax =plt.subplots(1,3, figsize=(14,3), dpi=300)
for mask, color, label in zip([mask_thin, mask_thick, mask_halo],['b','r','g'], ['thin', 'thick','Halo']):
    ax[0].scatter(V[mask],U[mask],s=0.2, alpha=0.5, c=color, label=label)
    ax[1].scatter(U[mask],W[mask],s=0.2, alpha=0.5, c=color, label=label)
    ax[2].scatter(V[mask],W[mask],s=0.2, alpha=0.5, c=color, label=label)
    
ax[0].scatter(0,0,s=5,c='k')
ax[0].set_xlabel("V")
ax[0].set_ylabel("U")
ax[1].scatter(0,0,s=5,c='k')
ax[1].set_xlabel("U")
ax[1].set_ylabel("W")
ax[2].scatter(0,0,s=5,c='k')
ax[2].set_xlabel("V")
ax[2].set_ylabel("W")
ax[2].legend()

not the super cool gradient you might have expected, that requires more careful binning of the data.

In [ ]:
V.max()

### The LSR frame
-------------------
To convert to LSR velocities, we need to subtract the solar motion from the measured velocities.

That can be automatically done with adding `.galacticlsr` after the coord objects.  In this ref frame, (0,0,0) in velocities, correspond to the velocity w.r.t LSR. 

In [ ]:
coord.GalacticLSR() #defaults in galactic lsr frame

Notice that v_bary matches with Schonrich et al. (2010) values

In [ ]:
st_n_lsr=st_n.galacticlsr
st_g_lsr= st_g.galacticlsr
st_n_lsr

In [ ]:
U,V,W = st_g_lsr.velocity.d_xyz
fig, ax =plt.subplots(1,3, figsize=(14,3), dpi=300)
ax[0].scatter(V,U,s=0.2)
ax[0].scatter(0,0,s=5,c='k')
ax[0].set_xlabel("V")
ax[0].set_ylabel("U")
ax[1].scatter(U,W,s=0.2)
ax[1].scatter(0,0,s=5,c='k')
ax[1].set_xlabel("U")
ax[1].set_ylabel("W")
ax[2].scatter(V,W,s=0.2)
ax[2].scatter(0,0,s=5,c='k')
ax[2].set_xlabel("V")
ax[2].set_ylabel("W")

Now all zero points are set to LSR velocities.

### apply_space_motion, Watch the stars dance!
---------------------------------------------------

Below is an astropy tutorial to visualize stellar motion by integrating velocity information with time. This section is copied from this [Astropy tutorial](https://learn.astropy.org/tutorials/gaia-visualization.html). 

In [ ]:
st_n_gal

In [ ]:
def star_plot(star_coords, vmin=None, vmax=None):
    """
    Take a list of stars as SkyCoord objects and build a figure showing the stars projected onto
    the plane of the sky using the Aitoff projection, and in three dimensional space centered
    on the Earth. In both plots the points are colored by distance (from Earth).

    Parameters
    ----------
    star_coords : Astropy SkyCoord object
        The coordinates of the objects to be plotted
    vmin, vmax : float
        Optional min and max values in pc for the colormap used to color the plot by distance.
        If not set they will be the minimum and maximum distances of star_coords.

    Returns
    -------
    response : matplotlib figure

    """

    # Set the min and max values for the colormap if not given
    if not vmin:
        vmin = star_coords.distance.min().value

    if not vmax:
        vmax = star_coords.distance.max().value

    # Initialize the figure
    fig = plt.figure(figsize=(10, 4),dpi=300)

    # Sky Projection plot
    ax1 = fig.add_subplot(1, 2, 1, projection="aitoff")  # Add left subplot

    # Turn off the axis ticks and labels, and turn on the plot grid
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    ax1.grid(True)

    # Plot the stars, colored by distance
    ax1.scatter(
        star_coords.ra.wrap_at(180 * u.deg).radian,
        star_coords.dec.radian,
        c=star_coords.distance,
        vmin=vmin,
        vmax=vmax,
        marker="*",
        s=200,
        cmap="plasma",
    )

    # 3D plot
    ax2 = fig.add_subplot(1, 2, 2, projection="3d")  # Add right subplot

    # Turn off axis ticks and labels
    ax2.set_xticklabels([])
    ax2.set_yticklabels([])
    ax2.set_zticklabels([])

    # Plotting the Earth as a black circle
    ax2.scatter([0], [0], [0], s=100, c="k", marker="o")

    # Plot the stars, colored by distance
    pc = ax2.scatter(
        star_coords.cartesian.x,
        star_coords.cartesian.y,
        star_coords.cartesian.z,
        c=star_coords.distance,
        s=200,
        vmin=vmin,
        vmax=vmax,
        marker="*",
        cmap="plasma",
    )

    # Adding the colorbar
    cbar = fig.colorbar(pc, shrink=0.6, location="right")
    cbar.ax.tick_params(labelsize=14)
    cbar.ax.set_ylabel("Distance (pc)", fontsize=18)

    # Remove extra space between the subplots
    fig.subplots_adjust(wspace=0)

    # This prevents getting an extra copy of the plot when you call the function
    plt.close()

    return fig

In [ ]:
import warnings  # So we can suppress expected warnings
import matplotlib.animation as animation

plt.rcParams["animation.html"] = (
    "jshtml"  # To make the animations render correctly in the notebook
)

In [ ]:
def star_animation(star_coords, evolution_time, steps, vmin=None, vmax=None):
    """
    Take a list of stars as a SkyCoord object and build an animation showing how the star's
    positions evolve over time.

    Two subplots are produced, the left showing the stars projected onto the plane of the sky
    using the Aitoff projection, and the right in three dimensional space centered on the Earth.
    In both plots the points are colored by distance (from Earth).

    Parameters
    ----------
    star_coords : Astropy SkyCoord object
        The coordinates of the objects you wish to plot
    evolution_time : float or Astropy Quantity
        The amount of time to evolve the stellar positions each step.
        If not specified as a quantity, years will be assumed.
    steps : int
        The number of time steps to take.
        The total amount of time the system will be eveolved for is evolution_time * steps
    vmin, vmax : float
        Optional min and max values for the colormap used to color the plot by distance.
        If not set they will be the minimum and maximum distances of the input stars at present time.

    Returns
    -------
    response : matplotlib figure

    """

    # Make sure the evolution time is in years
    if not isinstance(evolution_time, u.Quantity):
        evolution_time *= u.yr

    # Create array of time deltas (all in relation to present time)
    dt_array = np.linspace(0, evolution_time * steps, steps, endpoint=False)

    # If vmin/vmax were not set we need to set them (if they are not set at all,
    # each step of the animation will have different colormap boundaries).
    if not vmin:
        vmin = star_coords.distance.min().value

    if not vmax:
        vmax = star_coords.distance.max().value

    def animate(iteration, dt_array, aitoff_scatter, cube_scatter):
        """
        This function handles updating the plot for each frame of the animation.

        Parameters
        ----------
        iteration: int
            Current iteration (frame number)
        dt_array : Quantity
            Array of time deltas
        aitoff_scatter : matplotlib PathCollection
            A matplotlib collection object that holds the 2D (left) plot data points.
        cube_scatter : matplotlib Path3DCollection
            A matplotlib collection object that holds the 3D (right) plot data points.
        """

        with warnings.catch_warnings():
            # Projecting more than 5 years into the future gives a "dubious year" warning
            # due to the unpredictability of leap seconds, we suppress this warning becuase
            # we are not concerned with temporal accuracy to the second
            warnings.filterwarnings("ignore", message="ERFA function ")
            new_pos = star_coords.apply_space_motion(dt=dt_array[iteration])

        # set_offsets sets the point locations to the newly calculated coordinates
        aitoff_scatter.set_offsets(
            list(zip(new_pos.ra.wrap_at(180 * u.deg).radian, new_pos.dec.radian))
        )

        # set_array sets the colors of the points based on their new distances
        aitoff_scatter.set_array(new_pos.distance)

        # For the 3D plot we have to use the private property _offsets3d to update the point locations
        cube_scatter._offsets3d = (
            new_pos.cartesian.x,
            new_pos.cartesian.y,
            new_pos.cartesian.z,
        )

        # set_array sets the colors of the points based on their new distances
        cube_scatter.set_array(new_pos.distance)

        return (
            aitoff_scatter,
            cube_scatter,
        )

    # Call our star_plot function from the previous section to initialize the figure
    fig = star_plot(star_coords, vmin=vmin, vmax=vmax)

    # Get the matplotlib Collection objects that correspond to our sets of points
    aitoff_scatter = fig.axes[0].collections[0]
    cube_scatter = fig.axes[1].collections[1]

    # Build the animation
    ani = animation.FuncAnimation(
        fig,  # The figure we are animating
        animate,  # The function that updates the plot for each frame
        fargs=(dt_array, aitoff_scatter, cube_scatter),  # Args for the animate function
        frames=steps,  # The number of frames in our animation
        interval=100,
    )  # Delay between frames in milliseconds

    # This prevents getting an extra copy of the first frame of the plot when you call the function
    plt.close()

    return ani

In [ ]:
star_plot(st_n[:100])

In [ ]:
star_animation(st_n[:100], 10000*u.yr, 20)

### How fast is the sun rotating? Measuring the Oort's constants
-----------------------------------------------------------------------
Remember, we derived in class, the radial and transverse component of the velocities of the stars can be expressed as: 

$$ v_r= Ad \sin{2l} $$
$$ v_t= Ad \cos{2l} + Bd $$

Where A & B are Oort's constants, and they are:

$$ A= -\dfrac{1}{2}\left[ \left.\dfrac{d \Theta}{d R}\right|_{R_0} -\dfrac{\Theta_0}{R_0}  \right] $$
$$ B= -\dfrac{1}{2}\left[ \left.\dfrac{d \Theta}{d R}\right|_{R_0} +\dfrac{\Theta_0}{R_0}  \right] $$

If we can measure Oort's constants, finding rotation rate of sun ($\Theta_0$) is easy, as 
$$\Theta_0=A-B$$ 
And the change of rotation rate in the solar neighborhood: 

$$ \left.\dfrac{d \Theta}{d R}\right|_{R_0} = A+B $$

The values of A & B, from Carrol & Ostille are: 

$$A= 14.8 \pm 0.8 ~\mathrm{km~s^{-1}~kpc^{-1}}$$
$$B= -12.4 \pm 0.6 ~\mathrm{km~s^{-1}~kpc^{-1}}$$

Which are adopted from [Feast & Whitelock 1997](https://ui.adsabs.harvard.edu/abs/1997MNRAS.291..683F/abstract) *(Another result from Hipparcos)*

Now, in our dataset, we have all the info, $l$, $d$, $v_r$ and proper motion along $l$ and $b$ which we can convert to $v_t$.
Hence we can delve into measuring Oort's constant ourselves and find out how fast is the sun rotating! (and how fast $\Theta$ changes in solar neighborhood)  

We will use `st_g_gal` for this exercise

In [ ]:
#Plotting all the data
fig, ax =plt.subplots(figsize=(4,3),dpi=300)
ax.scatter(st_g_gal.l, st_g_gal.radial_velocity,s=2)

We see a rough sinusoidal variation indeed! But this is at all distances, so all sin 2l are superposed. Let's bin the data at various distances. 

In [ ]:
plt.hist(st_g_gal.distance.value, bins=np.logspace(0,4,30))
plt.xscale('log')

The distribution peaks at 1 kpc. Let's take all the stars within 900 pc to 1100 pc, and plot the vr vs l for them. (A lot of stars are in this region so should be fine I guess).

*Q: Can we still call it solar neighborhood?*

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def vr_fitter(l,A, d,**kwargs):
    return A*d*np.sin(2*l)

In [ ]:
dist_mask= ((st_g_gal.distance.value>900) & (st_g_gal.distance.value<1100))
dist_m=np.median(st_g_gal[dist_mask].distance.value)/1000.0  # Distance in kpc
l_min=np.min(st_g_gal[dist_mask].l.radian)
l_max=np.max(st_g_gal[dist_mask].l.radian)
lx=np.linspace(l_min,l_max,1000)

A_opt, dA = curve_fit( lambda l, A: vr_fitter(l, A, d=dist_m),
                       st_g_gal[dist_mask].l.radian,
                       st_g_gal[dist_mask].radial_velocity.value )

fig, ax =plt.subplots(figsize=(4,3),dpi=300)
ax.scatter(st_g_gal[dist_mask].l.radian, st_g_gal[dist_mask].radial_velocity,s=2)
ax.plot(lx, vr_fitter(lx, A_opt, d=dist_m), color='r')
#ax.scatter(st_g_gal[dist_mask].l.radian,vr_fitter(st_g_gal[dist_mask].l.radian,A=14.8,d=st_g_gal[dist_mask].distance.value/1000),color='orange',s=2)
print(f'A: {A_opt[0]}+/-{dA[0][0]}')

Therefore, our measured value for Oort's constant A: $12.18 \pm 2.48 ~\mathrm{km~s^{-1}~kpc^{-1}}$

You can try different distance binning and employ other sophisticated methods to fit the data.

#### Computing v_t
----------------------
If you remember, while deriving Oort's constant, we assumed all motions are confined in galactic plane only. Therefore, motion along b. which is going out of plane, we might discard, and only think about the motion along the galactic longitude, $v_l$

In [ ]:
vt= (st_g_gal.distance* st_g_gal.pm_l_cosb).to(u.km/u.s, u.dimensionless_angles())  #Transverse velocity on galactic plane

In [ ]:
def vt_fitter(l,B,A,d):
    return A*d*np.cos(2*l)+B*d

In [ ]:
B_opt, dB = curve_fit( lambda l, B: vt_fitter(l, B, A=12.18, d=dist_m),
                       st_g_gal[dist_mask].l.radian,
                       vt[dist_mask])
fig, ax =plt.subplots(dpi=300)
ax.scatter(st_g_gal[dist_mask].l.radian, vt[dist_mask],s=2)
ax.plot(lx, vt_fitter(lx, B_opt,A=12.18, d=dist_m), color='r')
ax.set_ylim(top=150)
print(f' B:{B_opt[0]} +/- {dB[0][0]}')
#pcov

Our measured value of B is: $B=-14.57 \pm 1.76 ~\mathrm{km~s^{-1}~kpc^{-1}}$

Therefore the solar rotation is:

In [ ]:
R0=8.1 # in kpc
Theta_0= (A_opt[0]-B_opt[0])*R0 #in km/s
Theta_0*u.km/u.s

Which is pretty close to the value 220 km/s mentioned in the book! What's our uncertainty here? 

Check the result with different distance binning.

### The Galactocentric frame
------------------------------
In all our discussion so far, origin of the coordinate system has been at the solar system barycenter. However, to understand Galactic kinematics & dynamics, it's often useful to use a frame centered at the Galactic center. coord.Galactocentric() is one such frame. 

In [ ]:
coord.Galactocentric() #The defaults

Notice that the default Galactocentric distance: $R_0=8.122 kpc$ and default galactocentric velocity of sun is(LSR+pecuiliar): galcen_v_sun=(12.9, 245.6, 7.78) km / s. On the other hand sun is located (z=20.8 pc) above the galactic plane. 

To convert to Galactocentric frame using defaults:

In [ ]:
st_g_galcen= st_g.galactocentric

In [ ]:
st_g

In [ ]:
#However, to compute it with changes in the input:
st_g_galcen= st_g.transform_to(coord.Galactocentric(z_sun=0*u.pc, galcen_distance=8.1*u.kpc))

### Visualizing with cylindrical representation

In [ ]:
st_g_rho, st_g_phi, st_g_z=st_g_galcen.cylindrical.rho, st_g_galcen.cylindrical.phi, st_g_galcen.cylindrical.z

In [ ]:
fig, ax= plt.subplots(figsize=(4,3), dpi=300, subplot_kw={'projection': 'polar'})
ax.scatter(st_g_phi.wrap_at(180 * u.deg).radian, st_g_rho, s=0.1, alpha=0.5)
ax.scatter(np.pi, 8100, s=5, c='yellow', marker='o' )
ax.set_yticks([2000,4000,8000,16000])
ax.set_yticklabels(['2','4','8','16'])
ax.set_xticklabels([])
ax.grid(True)
ax.set_title("Top view ")

In [ ]:
fig.savefig("galac_top_view_gaia.png",dpi=300, bbox_inches='tight')

In [ ]:
fig, ax= plt.subplots(figsize=(4,3), dpi=300)
ax.scatter(st_g_rho, st_g_z, s=0.1, alpha=0.5)
ax.set_xlabel(r"$\rho$")
ax.set_ylabel("z")
ax.set_title("side view")

## Computing galactic orbits with gala
---------------------------------------
For each of the 10000 stars we have a phase space point, i.e. (**x**, **v**). All these stars are moving in a gravitational potential composed of stars, gas & dust, galactic bulge and bar, dark matter halo. One can construct a mean gravitational potential that each star feels, and solve for the equation of motion to get the stellar orbits. 

`Gala` has in-built tools that are capable of doing that. Gala.potential, imported as `gp` here can provide various types of gravitational potential. They have a default MilkyWayPotential which we will use to compute the stellar orbits. 

You may also see the astropy tutorial [here](https://learn.astropy.org/tutorials/gaia-galactic-orbits.html), which I basically copied. 

In [ ]:
milky_way = gp.MilkyWayPotential()
milky_way

Other options are:

In [ ]:
w0 = gd.PhaseSpacePosition(st_g_galcen.cartesian)
w0

In [ ]:
orbits = H.integrate_orbit(w0, dt=1*u.Myr, t1=0*u.Myr, t2=500*u.Myr)

In [ ]:
fig = orbits[:,:5].plot()